## Import datasets

The dataset is obtained from Kaggle: "Airline Sentiment: Part 2" (https://www.kaggle.com/solegalli/airline-sentiment-part-2)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dataset/tweets.csv')
text = df['text']
sentiment = df['airline_sentiment']

## Data Preprocessing

In [4]:
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [22]:
num_words = 10000
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(text)

In [49]:
x_train_tokens = tokenizer.texts_to_sequences(text)
x_train_tokens = pad_sequences(x_train_tokens, maxlen=100, padding='pre', truncating='pre')

In [56]:
def one_hot_encoder(y):
    list_label = list(set(y))
    label = np.zeros([len(y), len(list_label)])
    for i in range(len(y)):
        label[i][list_label.index(y[i])] = 1
    return label

y_train = one_hot_encoder(sentiment)

In [5]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [61]:
model = Sequential()
max_tokens = 100
embedding_size = 8
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(3, activation='sigmoid'))
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
y_train = np.array(y_train)
model.fit(x_train_tokens, y_train)

/Users/fahmisalman/.conda/envs/Anaconda3.6/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
14640/14640 [==============================] - 121s 8ms/step - loss: 0.5315 - acc: 0.7585


In [9]:
import pandas as pd
import numpy as np
import re

In [10]:
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam

In [18]:
def preprocessing(text):
    text = re.sub('\n', '. ', text)
    text = text.lower()
    text = re.sub(r'[^a-z]', ' ', text)
    return text


def label_cleansing(text):
    text = re.sub('Dataset', '', text)
    text = re.sub('.txt', '', text)
    text = re.sub('/', '', text)
    text = re.sub(r'[^a-z]', '', text)
    return text


def one_hot_encoder(y):
    list_label = list(set(y))
    label = np.zeros([len(y), len(list_label)])
    for i in range(len(y)):
        label[i][list_label.index(y[i])] = 1
    return label


def fit(x_train_tokens, y_train, max_tokens=5000, embedding_size=8, num_words=10000, summary=False):

    model = Sequential()
    model.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size,
                        input_length=max_tokens,
                        name='layer_embedding'))
    # model.add(GRU(units=16, return_sequences=True))
    # model.add(GRU(units=8, return_sequences=True))
    model.add(GRU(units=64))
    model.add(Dense(3, activation='softmax'))
    optimizer = Adam(lr=1e-3)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    y_train = np.array(y_train)
    if summary:
        print(model.summary())
    model.fit(x_train_tokens, y_train, epochs=5)
    return model


In [19]:
df = pd.read_csv('dataset/tweets.csv')
text = df['text']
sentiment = df['airline_sentiment']

In [20]:
x_train = []
y_train = []

for i in range(len(text)):
    x_train.append(preprocessing(text[i]))
    y_train.append(label_cleansing(sentiment[i]))

num_words = 10000
max_tokens = 100
embedding_size = 256

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(x_train)

x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_train_tokens = pad_sequences(x_train_tokens, maxlen=max_tokens, padding='pre', truncating='pre')
y_train = one_hot_encoder(y_train)

model = fit(x_train_tokens, y_train, max_tokens=max_tokens,
            embedding_size=embedding_size,
            num_words=num_words,
            summary=True)

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 100, 256)          2560000   
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                61632     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 195       
Total params: 2,621,827
Trainable params: 2,621,827
Non-trainable params: 0
_________________________________________________________________
None


/Users/fahmisalman/.conda/envs/Anaconda3.6/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
14640/14640 [==============================] - 89s 6ms/step - loss: 0.4617 - acc: 0.7915
Epoch 2/5
14640/14640 [==============================] - 90s 6ms/step - loss: 0.3143 - acc: 0.8661
Epoch 3/5
14640/14640 [==============================] - 89s 6ms/step - loss: 0.2347 - acc: 0.9045
Epoch 4/5
14640/14640 [==============================] - 88s 6ms/step - loss: 0.1730 - acc: 0.9328
Epoch 5/5
14640/14640 [==============================] - 103s 7ms/step - loss: 0.1304 - acc: 0.9524
Saved model to disk
